In [ ]:
#
#ex1
import gymnasium as gym
import numpy as np

env = gym.make("Taxi-v3",render_mode="human")
state_size=env.observation_space.n
action_size=env.action_space.n

print("Espace d'actions :", env.action_space)
print("Espace d'observations :", env.observation_space)

observation, info = env.reset()

for _ in range(100):
    action = env.action_space.sample()  
    observation, reward, terminated, truncated, info = env.step(action)  
    
    if terminated or truncated:
        observation, info = env.reset() 

env.close()

Espace d'actions : Discrete(6)
Espace d'observations : Discrete(500)


: 

In [2]:
#ex2
import gymnasium as gym
import numpy as np

# Initialize the environment
env = gym.make("Taxi-v3",render_mode="human")

# Get the number of states and actions
state_size = env.observation_space.n
action_size = env.action_space.n

print(f"Number of states: {state_size}")
print(f"Number of actions: {action_size}")

# Run 20 episodes with random actions
for episode in range(20):
    state, info = env.reset()
    terminated = False
    truncated = False
    total_reward = 0
    
    print(f"\n=== Episode {episode + 1} ===")
    step = 0
    
    while not terminated and not truncated:
        # Take random action
        action = env.action_space.sample()
        
        # Execute the action
        next_state, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        
        # Display information
        print(f"Step {step}: State {state} -> Action {action} -> Reward {reward}")
        
        # Update state
        state = next_state
        step += 1
    
    print(f"Episode {episode + 1} finished after {step} steps. Total reward: {total_reward}")

env.close()

Number of states: 500
Number of actions: 6

=== Episode 1 ===
Step 0: State 13 -> Action 2 -> Reward -1
Step 1: State 33 -> Action 2 -> Reward -1
Step 2: State 33 -> Action 3 -> Reward -1
Step 3: State 13 -> Action 4 -> Reward -10
Step 4: State 13 -> Action 0 -> Reward -1
Step 5: State 113 -> Action 3 -> Reward -1
Step 6: State 113 -> Action 5 -> Reward -10
Step 7: State 113 -> Action 2 -> Reward -1
Step 8: State 133 -> Action 2 -> Reward -1
Step 9: State 133 -> Action 2 -> Reward -1
Step 10: State 133 -> Action 0 -> Reward -1
Step 11: State 233 -> Action 5 -> Reward -10
Step 12: State 233 -> Action 4 -> Reward -10
Step 13: State 233 -> Action 3 -> Reward -1
Step 14: State 213 -> Action 2 -> Reward -1
Step 15: State 233 -> Action 0 -> Reward -1
Step 16: State 333 -> Action 0 -> Reward -1
Step 17: State 433 -> Action 3 -> Reward -1
Step 18: State 433 -> Action 3 -> Reward -1
Step 19: State 433 -> Action 2 -> Reward -1
Step 20: State 453 -> Action 2 -> Reward -1
Step 21: State 453 -> Act

: 

In [ ]:
import gymnasium as gym
import numpy as np

# Initialize environment
env = gym.make("Taxi-v3")

# Hyperparameters
gamma = 0.99  # Discount factor
lr_policy = 0.1  # Learning rate for policy update
lr_value = 0.1  # Learning rate for value function update
clip_epsilon = 0.2  # PPO Clipping range
num_episodes = 100  # Number of training episodes

# Initialize policy table (equal probabilities) and value table
state_size = env.observation_space.n
action_size = env.action_space.n
policy_table = np.ones((state_size, action_size)) / action_size  # Ensure probabilities sum to 1
value_table = np.zeros(state_size)

def compute_discounted_rewards(rewards, gamma):
    """ Compute discounted rewards """
    discounted_rewards = np.zeros_like(rewards, dtype=np.float32)
    R = 0
    for t in reversed(range(len(rewards))):
        R = rewards[t] + gamma * R
        discounted_rewards[t] = R
    return discounted_rewards

for episode in range(num_episodes):
    state, _ = env.reset()
    episode_states = []
    episode_actions = []
    episode_rewards = []

    # Collect episode data
    done = False
    while not done:
        action = np.random.choice(action_size, p=policy_table[state])  # Sample action based on policy
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Store trajectory
        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)

        state = next_state

    # Compute discounted rewards
    discounted_rewards = compute_discounted_rewards(episode_rewards, gamma)

    # Compute advantage function A_t = R_t - V(s_t)
    advantages = discounted_rewards - value_table[episode_states]

    # PPO Policy Update (Clipped)
    for i, state in enumerate(episode_states):
        action = episode_actions[i]
        advantage = advantages[i]
        
        old_prob = policy_table[state, action]
        new_prob = old_prob * np.exp(lr_policy * advantage)

        # Apply PPO clipping
        clipped_prob = np.clip(new_prob, old_prob * (1 - clip_epsilon), old_prob * (1 + clip_epsilon))
        policy_table[state, action] = clipped_prob

        # **Ensure policy probabilities sum to 1**
        policy_table[state] /= np.sum(policy_table[state])  # Normalize the row

    # Update Value Function
    for i, state in enumerate(episode_states):
        value_table[state] += lr_value * (discounted_rewards[i] - value_table[state])

print("Training complete!")


Episode 0, Avg Reward: -830.00
Episode 50, Avg Reward: -770.14
Episode 100, Avg Reward: -755.60
Episode 150, Avg Reward: -747.44
Episode 200, Avg Reward: -761.88
Episode 250, Avg Reward: -719.14
Episode 300, Avg Reward: -794.90
Episode 350, Avg Reward: -766.36
Episode 400, Avg Reward: -775.96
Episode 450, Avg Reward: -756.44
Episode 500, Avg Reward: -759.08
Episode 550, Avg Reward: -771.16
Episode 600, Avg Reward: -774.52
Episode 650, Avg Reward: -771.68
Episode 700, Avg Reward: -744.14
Episode 750, Avg Reward: -766.84
Episode 800, Avg Reward: -750.62
Episode 850, Avg Reward: -767.90
Episode 900, Avg Reward: -749.32
Episode 950, Avg Reward: -751.66

Evaluation Completed: Average Reward = -379.10
Best Episode Reward: -200
Worst Episode Reward: -2000


In [39]:
#ex4
import gymnasium as gym
import numpy as np

# Initialize environment
env = gym.make("Taxi-v3")

# Number of evaluation episodes
num_eval_episodes = 20
total_rewards = []

for ep in range(num_eval_episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False

    while not done:
        # Select the best action based on the learned policy
        action = np.argmax(policy_table[state])  # Greedy action selection
        next_state, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        done = terminated or truncated
        state = next_state

    total_rewards.append(total_reward)

# Compute average performance
average_reward = np.mean(total_rewards)

print(f"Evaluation over {num_eval_episodes} episodes:")
print(f"Average Total Reward: {average_reward}")
print(f"All Episode Rewards: {total_rewards}")


Evaluation over 20 episodes:
Average Total Reward: -1005.95
All Episode Rewards: [-200, -200, -2000, -200, -1991, -2000, -200, -200, -1991, -2000, -1973, -200, -1973, -200, -200, -200, -200, -1991, -200, -2000]


In [ ]:
import gymnasium as gym
import numpy as np
from collections import deque

# Initialize environment
env = gym.make("Taxi-v3")

# Hyperparameters
gamma = 0.99          # Discount factor
lr_policy = 0.01      # Learning rate for policy
lr_value = 0.1        # Learning rate for value function
clip_epsilon = 0.2    # PPO clipping parameter
num_episodes = 1000   # Increased number of episodes
batch_size = 10       # Update after this many episodes

# Initialize tables
state_size = env.observation_space.n
action_size = env.action_space.n

# Use logits for numerical stability
policy_logits = np.random.randn(state_size, action_size) * 0.01
value_table = np.zeros(state_size)

# Experience buffers
states_buffer = deque(maxlen=10000)
actions_buffer = deque(maxlen=10000)
rewards_buffer = deque(maxlen=10000)
old_probs_buffer = deque(maxlen=10000)

def compute_returns(rewards, gamma):
    """Compute discounted returns"""
    returns = np.zeros_like(rewards)
    R = 0
    for t in reversed(range(len(rewards))):
        R = rewards[t] + gamma * R
        returns[t] = R
    return returns

def softmax(x):
    """Compute softmax values"""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# Training loop
episode_rewards = []

for episode in range(num_episodes):
    state, _ = env.reset()
    episode_reward = 0
    done = False
    
    # Collect trajectory
    while not done:
        # Get action probabilities
        probs = softmax(policy_logits[state])
        action = np.random.choice(action_size, p=probs)
        
        # Take action
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        
        # Store experience
        states_buffer.append(state)
        actions_buffer.append(action)
        rewards_buffer.append(reward)
        old_probs_buffer.append(probs[action])
        
        episode_reward += reward
        state = next_state
    
    episode_rewards.append(episode_reward)
    
    # Update every batch_size episodes
    if episode % batch_size == 0 and len(states_buffer) > 0:
        # Convert buffers to arrays
        states = np.array(states_buffer)
        actions = np.array(actions_buffer)
        rewards = np.array(rewards_buffer)
        old_probs = np.array(old_probs_buffer)
        
        # Compute returns and advantages
        returns = compute_returns(rewards, gamma)
        advantages = returns - value_table[states]
        
        # Normalize advantages
        advantages = (advantages - np.mean(advantages)) / (np.std(advantages) + 1e-8)
        
        # Get current probabilities
        current_probs = softmax(policy_logits[states])[np.arange(len(actions)), actions]
        
        # PPO policy update
        ratios = current_probs / old_probs
        clipped_ratios = np.clip(ratios, 1-clip_epsilon, 1+clip_epsilon)
        policy_loss = -np.minimum(ratios * advantages, clipped_ratios * advantages)
        
        # Update policy logits
        for idx, s in enumerate(states):
            policy_logits[s, actions[idx]] -= lr_policy * policy_loss[idx]
        
        # Update value function
        value_table[states] += lr_value * (returns - value_table[states])
        
        # Clear buffers
        states_buffer.clear()
        actions_buffer.clear()
        rewards_buffer.clear()
        old_probs_buffer.clear()
    
    # Print progress
    if episode % 50 == 0:
        avg_reward = np.mean(episode_rewards[-50:]) if episode >= 50 else np.mean(episode_rewards)
        print(f"Episode {episode}, Avg Reward: {avg_reward:.2f}")

# Evaluation
num_eval_episodes = 20
total_rewards = []

for ep in range(num_eval_episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False
    
    while not done:
        # Use deterministic policy
        probs = softmax(policy_logits[state])
        action = np.argmax(probs)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        total_reward += reward
        state = next_state
    
    total_rewards.append(total_reward)

average_reward = np.mean(total_rewards)
print(f"\nEvaluation Completed: Average Reward = {average_reward:.2f}")
print(f"Best Episode Reward: {max(total_rewards)}")
print(f"Worst Episode Reward: {min(total_rewards)}")

env.close()